In [1]:
!pip3 install kafka-python

import json
import random
from datetime import datetime
from time import sleep
from kafka import KafkaConsumer

А тут мы опишем наши консьюмеры, которые будут читать сообщения
Ниже базовая настройка консьюмера и несколько дополнительных. 
* one_record – для одиночного чтения сообщений
* ten_record – для вычитки пачками по 10 сообщений
* consumer_auto_offset_config – конфигурация для автоматического коммита (раз в 100 мс)
* consumer_manual_offset_config – конфигурация для ручного коммита сообщений

In [ ]:

consumer_base_config = {
    # Брокеры Kafka
    'bootstrap_servers': ['localhost:9093', 'localhost:9095', 'localhost:9097'],  # Список хостов и портов брокеров Kafka

    # (Де)сериализация ключей и значений
    'key_deserializer': lambda k: k.decode('utf-8') if k else None,  # Десериализация ключа из байтов
    'value_deserializer': lambda v: v.decode('utf-8') if v else None,  # Десериализация значения из байтов
    # Начало чтения сообщений
    'auto_offset_reset': 'earliest',  # 'earliest' - читать с самого начала, 'latest' - только новые
    # Таймаут для poll()
    'consumer_timeout_ms': 1000,  # Таймаут для ожидания сообщений (в миллисекундах)
    'max_poll_interval_ms': 300000,  # Максимальное время между вызовами poll (по умолчанию 5 минут)
    'fetch_min_bytes': 1,  # Минимальный объем данных за один запрос
    'fetch_max_wait_ms': 500,  # Максимальное время ожидания перед получением данных
}
consumer_auto_offset_config = {
    # Автокоммит смещений
    'enable_auto_commit': True,  # Включить автоматическое подтверждение обработки сообщений
    'auto_commit_interval_ms': 100,  # Интервал автокоммита (в миллисекундах)
    'group_id': 'auto-commit'
}
consumer_manual_offset_config = {
    # Автокоммит смещений
    'enable_auto_commit': False,  # Включить автоматическое подтверждение обработки сообщений
    'group_id': 'manual_commit'
}

consumer_one_record_config = {
    'max_poll_records': 1,  # Максимальное количество сообщений за раз
    # Группа консьюмеров
    'group_id': 'one_record'  # ID группы консьюмеров
}

consumer_ten_record_config = {
    'max_poll_records': 10,  # Максимальное количество сообщений за раз
    # Группа консьюмеров
    'group_id': 'ten_record'  # ID группы консьюмеров
}

In [19]:
from datetime import datetime

def kafka_base_consumer_example(config, topic):
    consumer = KafkaConsumer(topic, **config)
    print(f"Подключен к Kafka. Читаю сообщения из топика '{topic}'...")

    try:
        while True:
            # Читаем сообщения пачками
            messages = consumer.poll(timeout_ms=1000)  # Тайм-аут в миллисекундах
            if not messages:
                continue
            
            now = datetime.now()
            print(f"Время: {now}")
            for partition, msgs in messages.items():
                print(f"Получено {len(msgs)} сообщений из раздела {partition.partition} топика {partition.topic}")
                for msg in msgs:
                    print(f"Сообщение: {msg.value}, Ключ: {msg.key}, Смещение: {msg.offset}")
    except KeyboardInterrupt:
        print("Остановка консьюмера")
    finally:
        # Закрываем консьюмера
        consumer.close()

Попробуем запустить и посмотреть, что мы прочитали

In [ ]:
TEST_TOPIC = 'new-topic'
auto_commit_config = consumer_base_config | consumer_auto_offset_config
kafka_base_consumer_example(auto_commit_config, TEST_TOPIC)

Подключен к Kafka. Читаю сообщения из топика 'new-topic'...
Время: 2024-11-22 19:37:45.153772
Получено 2 сообщений из раздела 2 топика new-topic
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "0"}, Ключ: "0", Смещение: 38
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "3"}, Ключ: "3", Смещение: 39
Время: 2024-11-22 19:37:45.653714
Получено 5 сообщений из раздела 1 топика new-topic
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "1"}, Ключ: "1", Смещение: 71
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "4"}, Ключ: "4", Смещение: 72
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "6"}, Ключ: "6", Смещение: 73
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "9"}, Ключ: "9", Смещение: 74
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "10"}, Ключ: "10", Смещение: 75
Время: 2024-11-22 19:37:46.162313
Получено 5 сообщений из раз

А теперь давайте вернемся в kafka-producer и запустим Producer в 33 ячейке. Не забудьте перед этим остановить пред-ий консьюмер. Как вы видите, мы стали читать по несколько сообщений в пачке

In [ ]:
TEST_TOPIC = 'new-topic'

kafka_base_consumer_example(auto_commit_config, TEST_TOPIC)

А тут опишем консьюмер с ручным коммитом, который делает коммит в начале обработки каждой пачки 

In [23]:
from datetime import datetime

def kafka_manual_start_consumer_example(config, topic):
    config["group_id"] = "manual_commit_before_process"
    consumer = KafkaConsumer(topic, **config)
    print(f"Подключен к Kafka. Читаю сообщения из топика '{topic}'...")

    try:
        while True:
            # Читаем сообщения пачками
            messages = consumer.poll(timeout_ms=1000)  # Тайм-аут в миллисекундах
            if not messages:
                continue
            consumer.commit()
            now = datetime.now()
            print(f"Время: {now}")
            for partition, msgs in messages.items():
                print(f"Получено {len(msgs)} сообщений из раздела {partition.partition} топика {partition.topic}")
                for msg in msgs:
                    print(f"Сообщение: {msg.value}, Ключ: {msg.key}, Смещение: {msg.offset}")
            sleep(3)
    except KeyboardInterrupt:
        print("Остановка консьюмера")
    finally:
        # Закрываем консьюмер
        consumer.close()

А здесь консьюмер, который будет делать коммит после окончания обработки пачки сообщений

In [24]:
from datetime import datetime

def kafka_manual_end_consumer_example(config, topic):
    config["group_id"] = "manual_commit_after_process"
    consumer = KafkaConsumer(topic, **config)
    print(f"Подключен к Kafka. Читаю сообщения из топика '{topic}'...")
    try:
        while True:
            # Читаем сообщения пачками
            messages = consumer.poll(timeout_ms=1000)  # Тайм-аут в миллисекундах
            if not messages:
                continue
            now = datetime.now()
            print(f"Время: {now}")
            for partition, msgs in messages.items():
                print(f"Получено {len(msgs)} сообщений из раздела {partition.partition} топика {partition.topic}")
                for msg in msgs:
                    print(f"Сообщение: {msg.value}, Ключ: {msg.key}, Смещение: {msg.offset}")
            sleep(3)
            consumer.commit()
    except KeyboardInterrupt:
        print("Остановка консьюмера")
    finally:
        # Закрываем консьюмер
        consumer.close()

Давайте переключимся на 38-ую ячейку kafka-producer и запустим "вечный продьюсер, который раз в 0.5 секунд будет отправлять сообщение и увеличивать ключ).  
Дальше попробуем подключиться к нашему топику. Запустите следующую ячейку, а потом, спустя какое-то время остановите её, немного подождите и запустите следующую. Проверьте, не было ли повторной обработки сообщения (одинаковый ключ)

In [35]:
manual_config = consumer_base_config | consumer_manual_offset_config
ONE_PARTITION_TOPIC = 'one-partition'
print(manual_config)
kafka_manual_start_consumer_example(manual_config, ONE_PARTITION_TOPIC)

{'bootstrap_servers': ['localhost:9093', 'localhost:9095', 'localhost:9097'], 'key_deserializer': <function <lambda> at 0x108d521f0>, 'value_deserializer': <function <lambda> at 0x108d52dc0>, 'auto_offset_reset': 'earliest', 'consumer_timeout_ms': 1000, 'max_poll_interval_ms': 300000, 'fetch_min_bytes': 1, 'fetch_max_wait_ms': 500, 'enable_auto_commit': False, 'group_id': 'manual_commit'}
Подключен к Kafka. Читаю сообщения из топика 'one-partition'...
Время: 2024-11-22 19:56:34.416276
Получено 52 сообщений из раздела 0 топика one-partition
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "783"}, Ключ: "783", Смещение: 90
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "784"}, Ключ: "784", Смещение: 91
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "785"}, Ключ: "785", Смещение: 92
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "786"}, Ключ: "786", Смещение: 93
Сообщение: {"text": "MY_FIRST_MESS

In [36]:
kafka_manual_start_consumer_example(manual_config, ONE_PARTITION_TOPIC)

Подключен к Kafka. Читаю сообщения из топика 'one-partition'...
Время: 2024-11-22 19:56:41.656289
Получено 14 сообщений из раздела 0 топика one-partition
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "43"}, Ключ: "43", Смещение: 142
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "44"}, Ключ: "44", Смещение: 143
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "45"}, Ключ: "45", Смещение: 144
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "46"}, Ключ: "46", Смещение: 145
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "47"}, Ключ: "47", Смещение: 146
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "48"}, Ключ: "48", Смещение: 147
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "49"}, Ключ: "49", Смещение: 148
Сообщение: {"text": "MY_FIRST_MESSAGE", "number_value": 14.223, "key": "50"}, Ключ: "50", Смещение: 149
Сообщение: {"t

In [ ]:
kafka_manual_end_consumer_example(consumer_base_config | consumer_manual_offset_config, TEST_TOPIC)